In [161]:
import pandas as pd
import re

import tqdm 
from tqdm import tqdm, tqdm_notebook
from functools import reduce
from collections import Counter

import string
import scipy
from scipy import spatial, sparse

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem import SnowballStemmer 
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from functools import reduce
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder, BigramAssocMeasures, TrigramAssocMeasures

## Sample Submission

In [125]:
sample = pd.read_csv('sample_submission.csv')
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


## Data

In [126]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# train['part'] = 'train'
# test['part'] = 'test'

# data = pd.concat([train, test])

In [127]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [128]:
train.text[0]

'This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.'

In [129]:
def CleanData(train, test):
    c = Counter()
    for df in [train, test]:
        for ind, row in tqdm_notebook(df.iterrows(), total = df.shape[0], desc = 'Build punctuation dict'):
            c += Counter(re.sub('\w+','',row.text))

    dct = dict(c)
    dct.pop(' ')

    for i in dct.keys():
        dct[i]=''

    for df in [train, test]:
        for index, row in tqdm_notebook(df.iterrows(), total = df.shape[0], desc = 'Clean texts'):
            df.text[index] = reduce(lambda x, y: x.replace(y, dct[y]), dct, df.text[index])

    return train, test

In [130]:
ctrain, ctest = CleanData(train, test)

In [151]:
texts = list(ctrain['text'])+list(ctest['text'])

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens = [i.lower() for i in tokens if ( i not in string.punctuation+'—' )]
    stop_words = stopwords.words('english')
    tokens = [i for i in tokens if ( i not in stop_words )]
    return tokens

In [158]:
Vectorizer = CountVectorizer(tokenizer=tokenize)
%time Vectorizer.fit(texts)
%time train_vec = Vectorizer.transform(ctrain['text'])
%time test_vec = Vectorizer.transform(ctest['text'])

Wall time: 54.8 s
Wall time: 39.1 s
Wall time: 16.3 s


In [162]:
tfidf = TfidfTransformer(smooth_idf=False,sublinear_tf=False)
stk = sparse.vstack([train_vec,test_vec])
tfidf.fit(stk)

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [163]:
tfidf_train = tfidf.transform(train_vec)
tfidf_test = tfidf.transform(test_vec)

In [166]:
tfidf_train

<19579x28591 sparse matrix of type '<class 'numpy.float64'>'
	with 250061 stored elements in Compressed Sparse Row format>

In [167]:
import fasttext

In [168]:
fasttext.cbow()

TypeError: cbow() takes at least 2 positional arguments (1 given)

In [173]:
model = fasttext.skipgram(train.text[0])
print(model.words) # list of words in dictionary

TypeError: skipgram() takes at least 2 positional arguments (1 given)